In [137]:
import tweepy
from tweepy import OAuthHandler

consumer_key = 'fZ6WRlU8BVFQ6uPWlqvucD4Ht'
consumer_secret = 't1NkI8W5AnkQzaxAlrqjjwHyIFb4F9L84z8xdu9RDVHhlDymvp'
access_token = '888021590587428864-faOymuXz2oUt6ZxZSYnkJrlzwJkHMyX'
access_secret = '8NH5Ge3r80Q1F32tLtlVUyowYu88Q4NSfzEPEfakqyIzu'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [138]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    
    #Utility function to clean tweet text by removing links, special characters using simple regex statements.

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+) | (\d+)", " ", tweet).split())
 
def get_tweet_sentiment(tweet):

    #Utility function to classify sentiment of passed tweet using textblob's sentiment method
  
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [139]:
from afinn import Afinn
afinn = Afinn()

In [140]:
import pandas as pd
pd.set_option('display.max_colwidth', 1500)

def toDataFrame(tweets):
    
    #Data Frame Object
    DataSet = pd.DataFrame()

   # DataSet['userID'] = [tweet.user.id for tweet in tweets]
    DataSet['id'] = [tweet.id for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    #DataSet['source'] = [tweet.source for tweet in tweets]
    DataSet['tweetText'] = [tweet.text.replace('\n', '<br />') for tweet in tweets]
    DataSet['cleanText'] = [clean_tweet(tweet.text) for tweet in tweets]
    DataSet['sentiment_tb'] = [TextBlob(clean_tweet(tweet.text)).polarity for tweet in tweets]
    DataSet['sentiment_af'] = [afinn.score(clean_tweet(tweet.text)) for tweet in tweets]
    DataSet['retweet_count'] = [tweet.retweet_count for tweet in tweets]
    DataSet['favorite_count'] = [tweet.favorite_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]

    return DataSet

In [145]:
Results=[]
for tweet in tweepy.Cursor(api.search,
                              q='"JPMorgan"',
                              lang="en" 
                              #since = "2017-06-15",
                              #until = "2017-07-20"
                              #count =1000,
                              #result_type="recent",
                              ).items():
    if 'RT @' not in tweet.text:
        Results.append(tweet)
tweet_frame = toDataFrame(Results)
 #tweet_frame.to_csv("DataSet.csv",encoding='utf-8')       

In [144]:
Results=[]
for tweet in tweepy.Cursor(api.user_timeline, 
                           id="19643448",
                              #since = "2017-01-15",
                              #until = "2017-07-20"
                              #count =1000,
                              #result_type="recent",
                              ).items():
    if 'RT @' not in tweet.text:
        Results.append(tweet)
tweet_frame = toDataFrame(Results)
tweet_frame.shape

(2521, 10)

In [129]:
import datetime

tweet_frame['tweetDate']=  [d.date() for d in tweet_frame['tweetCreated']] 
tweet_frame['tweetDay']=  [d.day for d in tweet_frame['tweetDate']] 

def posFra(x):
    x['pFra'] = x[x['sentiment_af']>0].shape[0]/x.shape[0]
    x['nFra'] = x[x['sentiment_af']<0].shape[0]/x.shape[0]
    return x
tweet_frame=tweet_frame.groupby('tweetDay').apply(posFra)

In [130]:
tweet_frame.head(10)

tweet_frame.to_csv("DataSet.csv",encoding='utf-8')